In [51]:
import json
import os
import warnings

from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
from pathlib import Path
import requests
import yaml

In [68]:
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 350)

In [53]:
# Current utc, yeat, month and day
# utcnow = datetime.utcnow().strftime("%Y%m%d_%H%M")
dateoftoday = datetime.utcnow().strftime("%Y%m%d")
year = datetime.utcnow().strftime("%Y")
month = datetime.utcnow().strftime("%m")
day = datetime.utcnow().strftime("%d")

### Norrtälje : 18003

In [54]:
norrtalje = [
    939947, #Norrtälje
    939448, #Rö
    902860, #Håtö
    902869, #Aspö
    909994, #Vätö
    903691, #Boda
    902833, #Salnö
    902856, #Näset
    909981, #Singö
    909987, #Yxlan
    940299, #Rånäs
    940329, #Blidö
    910051, #Väddö
    902604, #Korgil
    902821, #Labben
    923691, #Sättra
    924065, #Björkö
    903687, #Ropnäs
    924056, #Svartnö
    940340, #Herräng
    903663, #Norröra
    903662, #Oxhalsö
    902601, #Östernäs
]

In [55]:
902873, #Urö
902855, #Addarsnäs
910127, #Fejan
923695, #Fogdö
903665, #Issjö
903673, #Backa
903731, #Granö
902597, #Utsund
902842, #Håknäs
902845, #Tjockö
902862, #Gräskö
902947, #Lervik
903690, #Skäret
902868, #Kolsvik
903723, #Storvik
902827, #Granhill
902847, #Edsviken


(902847,)

In [56]:
939947, #Norrtälje
939448, #Rö
902860, #Håtö
902869, #Aspö
909994, #Vätö
903691, #Boda
902833, #Salnö
902856, #Näset
909981, #Singö
909987, #Yxlan
940299, #Rånäs
940329, #Blidö
910051, #Väddö
902604, #Korgil
902821, #Labben
923691, #Sättra
924065, #Björkö
903687, #Ropnäs
924056, #Svartnö
940340, #Herräng
903663, #Norröra
903662, #Oxhalsö
902601, #Östernäs
902873, #Urö
902855, #Addarsnäs
910127, #Fejan
923695, #Fogdö
903665, #Issjö
903673, #Backa
903731, #Granö
902597, #Utsund
902842, #Håknäs
902845, #Tjockö
902862, #Gräskö
902947, #Lervik
903690, #Skäret
902868, #Kolsvik
903723, #Storvik
902827, #Granhill
902847, #Edsviken

(902847,)

In [58]:
ids = pd.read_parquet("../data/hemnet_location_ids_900000_933815.parquet.gzip")
ids.shape

(33816, 2)

In [59]:
ids

,hemnet_location_id,area
0,900000,"Gunnarskog tätort, Arvika kommun"
1,900001,"Lilla Stenby tätort, Ekerö kommun"
2,900002,"Starrkärr och Näs tätort, Stenungsunds kommun"
3,900003,"Månkarbo tätort, Tierps kommun"
4,900004,"Raksta tätort, Tyresö kommun"
...,...,...
33811,933811,"Bjärka-Säby Storängen, Linköpings kommun"
33812,933812,"Björkbacksvägen, Linköpings kommun"
33813,933813,"Björklida Björkmo, Linköpings kommun"
33814,933814,"Björklida Svensbo, Linköpings kommun"


In [60]:
ids.query("area == 'Sverige'")

,hemnet_location_id,area
268,900268,Sverige
269,900269,Sverige
270,900270,Sverige
271,900271,Sverige
272,900272,Sverige
...,...,...
24046,924046,Sverige
24047,924047,Sverige
24048,924048,Sverige
24082,924082,Sverige


In [63]:
ids[['Column1', 'Column2']] = ids['area'].str.split(',', 1, expand=True)
ids['Column1'] = ids['Column1'].str.strip()
ids['Column2'] = ids['Column2'].str.strip()
ids

,hemnet_location_id,area,Column1,Column2
0,900000,"Gunnarskog tätort, Arvika kommun",Gunnarskog tätort,Arvika kommun
1,900001,"Lilla Stenby tätort, Ekerö kommun",Lilla Stenby tätort,Ekerö kommun
2,900002,"Starrkärr och Näs tätort, Stenungsunds kommun",Starrkärr och Näs tätort,Stenungsunds kommun
3,900003,"Månkarbo tätort, Tierps kommun",Månkarbo tätort,Tierps kommun
4,900004,"Raksta tätort, Tyresö kommun",Raksta tätort,Tyresö kommun
...,...,...,...,...
33811,933811,"Bjärka-Säby Storängen, Linköpings kommun",Bjärka-Säby Storängen,Linköpings kommun
33812,933812,"Björkbacksvägen, Linköpings kommun",Björkbacksvägen,Linköpings kommun
33813,933813,"Björklida Björkmo, Linköpings kommun",Björklida Björkmo,Linköpings kommun
33814,933814,"Björklida Svensbo, Linköpings kommun",Björklida Svensbo,Linköpings kommun


In [81]:
ids["Column2"].value_counts()

Norrtälje kommun                    2858
Ydre kommun                         1811
Växjö kommun                         781
Linköpings kommun                    624
Haninge kommun                       523
Karlskrona kommun                    414
Hultsfreds kommun                    394
Malung-Sälens kommun                 378
Österåkers kommun                    354
Göteborgs kommun                     296
Älvdalens kommun                     286
Trelleborgs kommun                   281
Halmstads kommun                     263
Värmdö kommun                        258
Vellinge kommun                      251
Ekerö kommun                         245
Luleå kommun                         244
Stockholms kommun                    242
Falkenbergs kommun                   230
Sundbybergs kommun                   218
Söderköpings kommun                  212
Marks kommun                         208
Höganäs kommun                       202
Hudiksvalls kommun                   198
Kalix kommun    

In [69]:
all_norrtalje_ids = ids.query("Column2 == 'Norrtälje kommun'").copy()
all_norrtalje_ids

,hemnet_location_id,area,Column1,Column2
2594,902594,"Norrbygård, Norrtälje kommun",Norrbygård,Norrtälje kommun
2595,902595,"Fjällbostrand, Norrtälje kommun",Fjällbostrand,Norrtälje kommun
2596,902596,"Klämmersudd+Kvarnsand, Norrtälje kommun",Klämmersudd+Kvarnsand,Norrtälje kommun
2597,902597,"Utsund, Norrtälje kommun",Utsund,Norrtälje kommun
2598,902598,"Norrby udde, Norrtälje kommun",Norrby udde,Norrtälje kommun
...,...,...,...,...
26753,926753,"Stora Skålviksholmen, Norrtälje kommun",Stora Skålviksholmen,Norrtälje kommun
26756,926756,"Södra Skålviksstigen, Norrtälje kommun",Södra Skålviksstigen,Norrtälje kommun
26757,926757,"Vinö, Norrtälje kommun",Vinö,Norrtälje kommun
26844,926844,"Hamravägen, Norrtälje kommun",Hamravägen,Norrtälje kommun


In [76]:
all_norrtalje_ids[~all_norrtalje_ids['Column1'].str.endswith('vägen')].sample(50)

,hemnet_location_id,area,Column1,Column2
14498,914498,"Norrängshöjden, Norrtälje kommun",Norrängshöjden,Norrtälje kommun
13008,913008,"Skomakarbacken, Norrtälje kommun",Skomakarbacken,Norrtälje kommun
17646,917646,"Utholmen, Norrtälje kommun",Utholmen,Norrtälje kommun
18258,918258,"Solö Gullskär, Norrtälje kommun",Solö Gullskär,Norrtälje kommun
19775,919775,"Kroppåsviken, Norrtälje kommun",Kroppåsviken,Norrtälje kommun
10240,910240,"Aspsunds Byväg, Norrtälje kommun",Aspsunds Byväg,Norrtälje kommun
11074,911074,"Bålbro Gård, Norrtälje kommun",Bålbro Gård,Norrtälje kommun
13431,913431,"Söderängsholmarna, Norrtälje kommun",Söderängsholmarna,Norrtälje kommun
16437,916437,"Svenska Högarna, Norrtälje kommun",Svenska Högarna,Norrtälje kommun
13684,913684,"Räknö Kalvholmen, Norrtälje kommun",Räknö Kalvholmen,Norrtälje kommun


In [74]:
2858 - 1171

1687

In [ ]:
all_norrtalje_ids[~all_norrtalje_ids['Column1'].str.endswith('vägen')].sample(50)

In [78]:
substrings = ['backe', 'backen', 'gata', 'gatan', 'gården', 'stig', 'stigen', 'vägen', 'väg']
all_norrtalje_ids[~all_norrtalje_ids['Column1'].str.endswith(tuple(substrings))]

,hemnet_location_id,area,Column1,Column2
2594,902594,"Norrbygård, Norrtälje kommun",Norrbygård,Norrtälje kommun
2595,902595,"Fjällbostrand, Norrtälje kommun",Fjällbostrand,Norrtälje kommun
2596,902596,"Klämmersudd+Kvarnsand, Norrtälje kommun",Klämmersudd+Kvarnsand,Norrtälje kommun
2597,902597,"Utsund, Norrtälje kommun",Utsund,Norrtälje kommun
2598,902598,"Norrby udde, Norrtälje kommun",Norrby udde,Norrtälje kommun
...,...,...,...,...
26749,926749,"Själbottna Nötholmen, Norrtälje kommun",Själbottna Nötholmen,Norrtälje kommun
26750,926750,"Själbottna Storudden, Norrtälje kommun",Själbottna Storudden,Norrtälje kommun
26751,926751,"Smedsholmen, Norrtälje kommun",Smedsholmen,Norrtälje kommun
26753,926753,"Stora Skålviksholmen, Norrtälje kommun",Stora Skålviksholmen,Norrtälje kommun


In [79]:
all_norrtalje_ids[~all_norrtalje_ids['Column1'].str.endswith(tuple(substrings))].sample(50)

,hemnet_location_id,area,Column1,Column2
15530,915530,"Rådmansö Mellanören, Norrtälje kommun",Rådmansö Mellanören,Norrtälje kommun
10842,910842,"Östernäs Täckö, Norrtälje kommun",Östernäs Täckö,Norrtälje kommun
23527,923527,"Löparö Gistplats, Norrtälje kommun",Löparö Gistplats,Norrtälje kommun
19467,919467,"Stor-Högskär, Norrtälje kommun",Stor-Högskär,Norrtälje kommun
22932,922932,"Lindbäcken Granöslingan, Norrtälje kommun",Lindbäcken Granöslingan,Norrtälje kommun
18310,918310,"Uröhöjden, Norrtälje kommun",Uröhöjden,Norrtälje kommun
17785,917785,"Rödlöga Långskär, Norrtälje kommun",Rödlöga Långskär,Norrtälje kommun
2841,902841,"Långängen, Norrtälje kommun",Långängen,Norrtälje kommun
23261,923261,"Lambskär, Norrtälje kommun",Lambskär,Norrtälje kommun
18252,918252,"Fjärdskärsören, Norrtälje kommun",Fjärdskärsören,Norrtälje kommun


In [ ]:
all_norrtalje_ids[all_norrtalje_ids['Column1'].str.endswith('vägen')]

In [ ]:
# Filter the DataFrame based on 'Column1' values ending with "vägen"
filtered_df = df[df]

# Display the filtered DataFrame
print(filtered_df)

In [ ]:
# Loop thruogh all the areas one by one
norrtalje_df_list = []

for area_code in norrtalje:
    print(area_code)

    # Start the process of scaring
    url = "https://www.hemnet.se/bostader"
    params= {
        #"housing_form_groups":"apartments",
        "location_ids":area_code,
        #"item_types":"bostadsratt",
        "rooms_min":0,
        "living_area_min":0,
        "new_construction":"include"
    }
    payload={}
    headers = {"User-Agent": "Mozilla/5.0"
    }

    response = requests.request(
        "GET",
        url,
        headers=headers,
        data=payload,
        params=params
    )

    soup = BeautifulSoup(response.content, "html.parser")
    map_results=soup.find(id="results-map")
    initial_data=map_results.attrs["data-initial-data"]
    json_data=json.loads(initial_data)

    url = "https://www.hemnet.se/bostader/search/"+json_data["search_key"]

    params= {}
    payload={}

    headers = {"User-Agent": "Mozilla/5.0"}

    response = requests.request(
        "GET",
        url,
        headers=headers,
        data=payload,
        params=params
    )

    r_json = response.json()
    r_properties = r_json["properties"]

    # Store all the listings in that area as a dataframe
    df = pd.json_normalize(
        r_properties,
        max_level=1
    )

    print(f"{df.shape}")

    # Enrich the df with info about internal area naming and date
    df["area"] = "norrtälje"
    df["date"] = dateoftoday

    norrtalje_df_list.append(df)

In [ ]:
norrtalje_complete_df = pd.concat(norrtalje_df_list)
norrtalje_complete_df.drop_duplicates(subset="id", inplace=True)
norrtalje_complete_df.shape

In [ ]:
norrtalje_complete_df.reset_index(inplace=True)
norrtalje_complete_df.drop("index", axis=1, inplace=True)
norrtalje_complete_df

In [ ]:
norrtalje_complete_df.to_parquet("../data/stockholms_lan/2023/05/29/norrtalje.parquet.gzip", compression="gzip")